In [1]:
from math import *
from leg import *
from body import *
from control import *
import numpy as np

In [2]:
b = Body()
c = Control(b)
c.build_quad()
c.walk(2, 0)

2.5 [[ 1.   0.   0.   0. ]
 [ 0.   1.   0.   0. ]
 [ 0.   0.   1.   0. ]
 [ 1.   0.  -2.5  1. ]]


###### 